In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time
print("Package loaded!!")

Package loaded!!


In [2]:
_data_dir = "./mnist/"

mnist = input_data.read_data_sets(_data_dir, one_hot=True)
print("Data loaded!!")

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Data loaded!!


In [3]:
img_test = mnist.test.images
label_test = mnist.test.labels

NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE  # 784
NUM_HIDDEN1_NODE = 256
NUM_HIDDEN2_NODE = 128

stddev = 0.1



In [4]:
def xavier_init(input_size, output_size):
    stddev = tf.sqrt(2.0 / (input_size + out_size))
    return tf.random_norm([input_size, output_size], stddev=stddev)

def linear(data_in, output_size, name="linear"):
    with tf.variable_scope(name):
        print data_in.get_shape()[-1], type(data_in.get_shape()[-1])
        input_size = data_in.get_shape().as_list()[-1]
        #weights = tf.Variable(xavier_init(input_size, output_size), name="weights")
        weights = tf.get_variable("weights", shape=[input_size, output_size], initializer=tf.contrib.layers.xavier_initializer())
        # weights = tf.Variable(tf.zeros([data_in.get_shape()[-1], output_size]), name="weights")  
        biases = tf.Variable(tf.zeros(output_size), name="biases")
        logits = tf.nn.xw_plus_b(data_in, weights, biases, name="logits")
        activation = tf.nn.sigmoid(logits, name="act")
        return logits, activation


def inference_mlp1(data_in, num_hidden_node):
    _, h1 = linear(data_in, num_hidden_node, name="linear1")
    logits, _ = linear(h1, NUM_CLASSES, name="linear2")
    return logits


def inference_mlp2(data_in, num_hidden1_node, num_hidden2_node):
    _, h1 = linear(data_in, num_hidden1_node, name="linear1")
    _, h2 = linear(h1, num_hidden2_node, name="linear2")
    logits, _ = linear(h2, NUM_CLASSES, name="linear3")
    return logits


def cost(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))


def training(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train = optimizer.minimize(loss, global_step=global_step)
    return train


def evaluation(logits, labels):
    pred = tf.equal(tf.argmax(tf.nn.softmax(logits), 1), tf.argmax(labels, 1))
    accr = tf.reduce_mean(tf.cast(pred, "float"))

    
    return accr

In [5]:
mnist_graph = tf.Graph()

with mnist_graph.as_default():
    learning_rate = 0.001
    x = tf.placeholder("float", [None, IMAGE_PIXELS])
    y = tf.placeholder("float", [None, NUM_CLASSES])
    #logits = inference_mlp1(x, NUM_HIDDEN1_NODE)
    logits = inference_mlp2(x, NUM_HIDDEN1_NODE, NUM_HIDDEN2_NODE)
    loss = cost(logits, y)
    train = training(loss, learning_rate=learning_rate)
    eval_correct = evaluation(logits, y)

    init = tf.initialize_all_variables()

    n_samples = mnist.train.num_examples
    batch_size = 200
    total_batch = int(n_samples / batch_size)
    start = time.time()
    total_epoch = 50

# SESSION
with tf.Session(graph=mnist_graph) as sess:
    sess.run(init)
    # MINI-BATCH LEARNING
    for epoch in range(total_epoch + 1):
        avg_cost = 0.
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed = {x: batch_xs, y: batch_ys}
            sess.run(train, feed)
            avg_cost += sess.run(loss, feed) / batch_size

        # DISPLAY
        if epoch % 5 == 0:
            feeds_train = {x: batch_xs, y: batch_ys}
            feeds_test = {x: img_test, y: label_test}
            train_acc = sess.run(eval_correct, feeds_train)
            test_acc = sess.run(eval_correct, feeds_test)
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f"
                   % (epoch, total_epoch, avg_cost, train_acc, test_acc))
print ("DONE")
print time.time() - start

784 <class 'tensorflow.python.framework.tensor_shape.Dimension'>
256 <class 'tensorflow.python.framework.tensor_shape.Dimension'>
128 <class 'tensorflow.python.framework.tensor_shape.Dimension'>
Epoch: 000/050 cost: 1.034192411 train_acc: 0.965 test_acc: 0.911
Epoch: 005/050 cost: 0.166828964 train_acc: 0.935 test_acc: 0.962
Epoch: 010/050 cost: 0.075388803 train_acc: 0.970 test_acc: 0.973
Epoch: 015/050 cost: 0.034190502 train_acc: 1.000 test_acc: 0.978
Epoch: 020/050 cost: 0.014784829 train_acc: 0.990 test_acc: 0.979
Epoch: 025/050 cost: 0.005446980 train_acc: 1.000 test_acc: 0.979
Epoch: 030/050 cost: 0.002131663 train_acc: 1.000 test_acc: 0.980
Epoch: 035/050 cost: 0.000903715 train_acc: 1.000 test_acc: 0.981
Epoch: 040/050 cost: 0.001787900 train_acc: 1.000 test_acc: 0.979
Epoch: 045/050 cost: 0.000314917 train_acc: 1.000 test_acc: 0.981
Epoch: 050/050 cost: 0.000198209 train_acc: 1.000 test_acc: 0.981
DONE
50.1449120045
